In [1]:
from dotenv import load_dotenv 
from azure.identity import DeviceCodeCredential
from msgraph import GraphServiceClient
from msgraph.generated.users.item.messages.messages_request_builder import MessagesRequestBuilder
from msgraph.generated.search.query.query_post_request_body import QueryPostRequestBody
from msgraph.generated.search.query.query_request_builder import QueryRequestBuilder
from msgraph.generated.search.search_request_builder import SearchRequestBuilder
from msgraph.generated.models.search_request import SearchRequest
from msgraph.generated.models.entity_type import EntityType
from msgraph.generated.models.search_query import SearchQuery
from msgraph.generated.models.search_response import SearchResponse
import os
import json
import asyncio 
import uuid

In [2]:
from typing import Tuple

import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding

In [4]:
kernel = sk.Kernel()
# Configure AI service used by the kernel
api_key, org_id = sk.openai_settings_from_dot_env()
#kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))
kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-4", api_key, org_id))
kernel.add_text_embedding_generation_service("ada", OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id))

kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
kernel.import_skill(sk.core_skills.TextMemorySkill())

{'recall': <semantic_kernel.orchestration.sk_function.SKFunction at 0x2204d4f4050>,
 'save': <semantic_kernel.orchestration.sk_function.SKFunction at 0x2204d4fc590>}

In [5]:
load_dotenv()

True

In [6]:
COLLECTION = "ESPC23"

In [8]:
TENANT_ID = os.getenv("TENANT_ID")
CLIENT_ID = os.getenv("CLIENT_ID")
graphUserScopes = os.getenv("graphUserScopes").split(' ')
gabby = '9c72876c-b0ec-41b3-8079-8ebe216daa34'
print(graphUserScopes)

['User.Read', 'Mail.ReadWrite', 'Chat.ReadWrite', 'Calendars.Read', 'Files.Read.All', 'Sites.Read.All']


In [9]:
credential = DeviceCodeCredential(client_id=CLIENT_ID, tenant_id=TENANT_ID) 
scopes = graphUserScopes
client = GraphServiceClient(credentials=credential, scopes=scopes)

In [10]:
#this will get the client
result = await client.me.get()

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BNVS8A36H to authenticate.


In [11]:
print(result.display_name)

Gabby Gabgab


## Do the Query and Get the Intent of the User Asking the Question (UserPrompt)

In [12]:
# note: Using a File to Run these Plugins instead of Inline to 
# show a different way
plugins_directory = "Plugins/"
intentFunctions = kernel.import_semantic_skill_from_directory(plugins_directory, "HoldMyHandPlugin")

getIntentFunction = intentFunctions["GetIntent"]

In [13]:
userIntent = getIntentFunction("What did I accomplish this weekend with Gabby in Antwerp regarding our ESPC demo?")

In [14]:
subject_data = json.loads(str(userIntent))
print(subject_data)

{'subject': 'Gabby', 'action': 'know what was accomplished during the ESPC demo in Antwerp'}


In [15]:
subject = subject_data['subject']
action = subject_data['action']

In [16]:
print(subject)

Gabby


In [17]:
print(action)

know what was accomplished during the ESPC demo in Antwerp


## Work with Teams(Chat) Search Query Endpoint

In [18]:
request_body = QueryPostRequestBody(
	requests = [
		SearchRequest(
			entity_types = [
				EntityType.ChatMessage
			],
			query = SearchQuery(
				query_string = subject,
			),
		),
	],
)

chat_result = await client.search.query.post(request_body)

OverflowError: Python int too large to convert to C long

In [19]:
chat_data = chat_result.value

NameError: name 'chat_result' is not defined

In [20]:
##Testing with GUID
search_response = chat_data[0] 

# Initialize an empty list to store summaries, chat IDs, and generated GUIDs
summaries_chat_ids_and_guids = []

# Iterate over hits_containers in the SearchResponse object
for hits_container in search_response.hits_containers:
    # Iterate over each hit in the container
    for hit in hits_container.hits:
        # Extract the summary
        summary = hit.summary

        # Extract the chatId from additional_data
        chat_id = hit.resource.additional_data.get('chatId', 'Unknown')

        # Generate a unique GUID for each hit
        guid = str(uuid.uuid4())

        # Append the chatId, summary, and GUID as a tuple to the list
        summaries_chat_ids_and_guids.append((guid, summary))

# Now 'summaries_chat_ids_and_guids' contains the chatId, summary, and a unique GUID for each search hit
for item in summaries_chat_ids_and_guids:
    await kernel.memory.save_information_async("ESPC23", id=guid, text=summary)
    print(item)

                                            

NameError: name 'chat_data' is not defined

In [21]:
print(summaries_chat_ids_and_guids)

NameError: name 'summaries_chat_ids_and_guids' is not defined

## Work with Drives(Files) Search Query Endpoint

In [22]:
request_body = QueryPostRequestBody(
	requests = [
		SearchRequest(
			entity_types = [
				EntityType.DriveItem
			],
			query = SearchQuery(
				query_string = subject,
			),
		),
	],
)

drive_result = await client.search.query.post(request_body)

In [23]:
drive_data = drive_result.value
#print(drive_data)

In [24]:
try:
    drive_search_response = drive_data[0]  # Assuming there's only one SearchResponse object in the list

    # Initialize an empty list to store summaries, from information, and generated GUIDs
    drive_summary_from_and_guids = []
    drive_narrative = []

    if drive_search_response:
        # Iterate over hits_containers in the SearchResponse object
        for drive_hits_container in drive_search_response.hits_containers:
            # Iterate over each hit in the container
            for drive_hit in drive_hits_container.hits:
                # Extract the summary
                drive_summary = drive_hit.summary

                # Extract the 'from' information from additional_data
                from_data = drive_hit.resource.additional_data.get('from', {}).get('emailAddress', {})
                from_name = from_data.get('name', 'Unknown')
                from_email = from_data.get('address', 'Unknown')
                file_name = getattr(drive_hit.resource, 'name', 'Unknown')
                user_email = from_data.get('email', 'Unknown')

                # Generate a unique GUID for each hit
                drive_guid = str(uuid.uuid4())

                # Append the summary, 'from' information, and GUID as a tuple to the list
                drive_summary_from_and_guids.append((drive_guid, drive_summary, from_name, from_email))
                drive_narrative.append((drive_guid, file_name))
except Exception as e:
    print(f"An error occurred: {e}")

# Now 'drive_summary_from_and_guids' contains the summary, from information, and a unique GUID for each search hit
for item in drive_narrative:
    await kernel.memory.save_information_async("ESPC23", id=drive_guid, text=file_name)
    print(item)



('5a0916f7-383f-4752-a942-5896bfa7a5f2', 'CollabSummit2024_GabbySpeaking.docx')
('a2b8cd58-8d6a-40d7-8187-a12431d684cd', 'Apps')
('4ef59747-696d-45fd-82ae-a57b50d44897', 'ProjectRoundtreeStatusReport.docx')
('29feceff-b445-4d8c-a3ba-2de9318a0cc8', 'HowToGainAccessToStudyGroup.docx')


In [25]:
print(drive_narrative)

[('5a0916f7-383f-4752-a942-5896bfa7a5f2', 'CollabSummit2024_GabbySpeaking.docx'), ('a2b8cd58-8d6a-40d7-8187-a12431d684cd', 'Apps'), ('4ef59747-696d-45fd-82ae-a57b50d44897', 'ProjectRoundtreeStatusReport.docx'), ('29feceff-b445-4d8c-a3ba-2de9318a0cc8', 'HowToGainAccessToStudyGroup.docx')]


## Work with Messages(Email) Search Query Endpoint

In [26]:
request_body = QueryPostRequestBody(
	requests = [
		SearchRequest(
			entity_types = [
				EntityType.Message
			],
			query = SearchQuery(
				query_string = subject,
			),
		),
	],
)

mail_result = await client.search.query.post(request_body)

In [27]:
mail_data = mail_result.value

In [28]:
try:
    mail_search_response = mail_data[0]  # Assuming there's only one SearchResponse object in the list

    # Initialize an empty list to store summaries, from information, and generated GUIDs
    mail_summary_from_and_guids = []

    if mail_search_response:
        # Iterate over hits_containers in the SearchResponse object
        for mail_hits_container in mail_search_response.hits_containers:
            # Iterate over each hit in the container
            for mail_hit in mail_hits_container.hits:
                # Extract the summary
                mail_summary = mail_hit.summary

                # Extract the 'from' information from additional_data
                from_data = mail_hit.resource.additional_data.get('from', {}).get('emailAddress', {})
                from_name = from_data.get('name', 'Unknown')
                from_email = from_data.get('address', 'Unknown')

                # Generate a unique GUID for each hit
                guid = str(uuid.uuid4())

                # Append the summary, 'from' information, and GUID as a tuple to the list
                mail_summary_from_and_guids.append((guid, mail_summary, from_name, from_email))
                mailconstruct = mail_summary + " from " + from_name
except Exception as e:
    print(f"An error occurred: {e}")

# Now 'mail_summary_from_and_guids' contains the summary, from information, and a unique GUID for each search hit
for item in mail_summary_from_and_guids:
    await kernel.memory.save_information_async("ESPC23", id=guid, text=mailconstruct)
    print(item)

('78325efe-7169-40a9-87e8-c22975338ef7', "Fabian Williams shared a file with you  Here's the document that Fabian Williams shared with you.   ProjectRoundtreeStatusReport   This link will work for anyone.  Open   Privacy...", 'Unknown', 'Unknown')
('c1b48f9c-3063-4e7d-835f-01704008a0bf', "Fabian Williams shared a file with you   Here's the document that Fabian Williams shared with you.   HowToGainAccessToStudyGroup  1 min to read this.   This link will work for...", 'Unknown', 'Unknown')
('630132f0-98ed-414b-a033-95ca62ffb4ec', 'From: Microsoft <microsoft-noreply@microsoft.com> Sent: Thursday, November 9, 2023 8:15 PM To: Fabian Williams <fabian@fabster.onmicrosoft.com> Subject: Your Microsoft invoice...', 'Unknown', 'Unknown')
('bf990692-bbe9-4ef6-b282-a18436a7eabe', 'Private to you Hi, <c0>Gabby</c0> Gabgab, Welcome to your new digest from Microsoft Viva Work smarter with Microsoft Viva Insights Explore insights on your work patterns in areas including...', 'Unknown', 'Unknown')
('c4

## Query and Read All 3 Payloads from Semantic Memory ONLY

In [29]:
async def search_memory_examples(kernel: sk.Kernel) -> None:
    questions = [
        "What did I accomplish this weekend with Gabby in Antwerp regarding our ESPC demo?",
        "What ESPC23 artifacts do I have?",
    ]

    for question in questions:
        results = await kernel.memory.search_async("ESPC23", question, limit=3)
        for result in results:
            print(f"Question: {question}")
            print(f"Answer: {result.text}\nRelevance: {result.relevance}\n")

In [30]:
await search_memory_examples(kernel)

Question: What did I accomplish this weekend with Gabby in Antwerp regarding our ESPC demo?
Answer: Ok, So make sure you know your lines, study your answers to the questions. Get your demo ready to go. MOST IMPORTANT: do all the sacrifices to the demo gods!!! Dad. from Unknown
Relevance: 0.7582602447035458

Question: What did I accomplish this weekend with Gabby in Antwerp regarding our ESPC demo?
Answer: HowToGainAccessToStudyGroup.docx
Relevance: 0.7546709696624034

Question: What ESPC23 artifacts do I have?
Answer: HowToGainAccessToStudyGroup.docx
Relevance: 0.7296164736930804

Question: What ESPC23 artifacts do I have?
Answer: Ok, So make sure you know your lines, study your answers to the questions. Get your demo ready to go. MOST IMPORTANT: do all the sacrifices to the demo gods!!! Dad. from Unknown
Relevance: 0.7186311721156557



## Setup RAG Function

In [31]:
print(COLLECTION)

ESPC23


In [33]:
async def setup_RAG(kernel: sk.Kernel) -> Tuple[sk.SKFunctionBase, sk.SKContext]:
    sk_prompt = """
    You are a friendly and very talkative ChatBot. 
    
    {{$history}}
    Answer the {{$user_input}}. You may use information provided in {{$retreived_context}} it may be useful. 
    Use information only from {{$retreived_context}} otherwise say 'Nothing came back from Microsoft Graph'. 
    Do not start off the response with I'm sorry
    ChatBot:""".strip()

    rag_func = kernel.create_semantic_function(sk_prompt, max_tokens=2000, temperature=0.1)

    context = kernel.create_new_context()

    return rag_func, context

In [35]:
async def RAG(kernel: sk.Kernel, rag_func: sk.SKFunctionBase, context: sk.SKContext) -> bool:

    user_input = input("User:> ")
    context["user_input"] = user_input
    #Retrieve
    result = await kernel.memory.search_async(COLLECTION,context["user_input"])

    
    if result[0].text:
        context["retreived_context"] = result[0].text
    
    #Then generate
    answer = await kernel.run_async(rag_func, input_vars=context.variables)

    print(f"\n\u001b[34mChatBot:> {answer}\u001b[0m \n\n\033[1;32m Source: {context['retreived_context']}\n Relevance: {result[0].relevance}\u001b[0m \n")
    return False

In [36]:
print("Simulate Chat experience with SK and RAG")
rag_func, context = await setup_RAG(kernel)

Simulate Chat experience with SK and RAG


In [37]:
print("Ask a question (type 'exit' to exit):\n")
answer = await RAG(kernel, rag_func, context)

Ask a question (type 'exit' to exit):

User:> What do I need to do to prepare for my ESPC demo

ChatBot:> To prepare for your ESPC demo, you should first ensure that you are well-versed with your lines. This means understanding the content you are going to present and being able to deliver it confidently. 

Next, you should study your answers to the questions. This could mean anticipating what questions might be asked and preparing your responses. It's always good to be prepared for any potential questions that may come your way during the demo.

Then, get your demo ready to go. This could involve setting up any necessary equipment, ensuring your presentation slides are in order, and making sure all technical aspects are functioning correctly.

Lastly, and most importantly, don't forget to do all the sacrifices to the demo gods! This is a humorous way of saying that you should do everything you can to ensure your demo goes smoothly. This could involve double-checking everything, doing 

### User Prompts that work

In [38]:
# GREAT
# What do I need to do to prepare for my ESPC demo
# GOOD
# What did I accomplish this weekend with Gabby in Antwerp regarding our ESPC demo?
# Where did I go this weekend and what did I do
# 
#